# MulticelSeqFS

Combine multiple Dcels together in a sequence. (No overlay.)

In [1]:
from fs.base import FS
from fs.info import Info
import six
from fs.errors import ResourceNotFound, DirectoryExpected
from copy import deepcopy
from MulticelFS import MulticelFS

def mergehostinfo(A,B,out=None):
    pass

def mergeinfo(A,B,out=None):
    # type: (dict, dict, dict) -> dict()

    if out is None:
        _out = dict()
    else:
        _out = out
    
    if A == dict():
        return deepcopy(B)
    
    for k,v in A.items():
        if k in B:
            vB = B[k]
            if type(v) is dict and type(vB) is dict:
                _out[k] = dict()
                mergeinfo(v,vB,_out[k])
            elif type(v) is list and type(vB) is list:
                #quick fix
                #todo: upgrade w mergehostinfo()
                _out[k] = list(set(v)|set(vB))
            else:
                _out[k] = B[k]
        else:
            _out[k] = A[k]
    for k,v in B.items():
        if not k in _out:
            _out[k] = B[k]
    return _out

class MulticelSeqFS(MulticelFS):
    def __init__(self, dcels):
        super().__init__(dcels)
    
    def listdir(self,path): # Get a list of resources in a directory.
        lsprep = list()
        for ea in self.dcels:
            try:
                lsprep.extend(
                    ea.listdir(path)
                )
            except ResourceNotFound:
                pass
            except DirectoryExpected:
                pass
        return lsprep

    
    # non-FS API specific to MultiCelFS
    #def add_dcel(self, dcel):
    #def remove_dcel(self, dcel):
    # def get_dcel_by_host(self,hostname):


In [7]:
from Dcel import Dcel
from blackstrap import BlackstrapFS

BlackstrapFS.initHost('localhost')
BlackstrapFS.addShare('images/fs','fs')
BlackstrapFS.addShare('images/fs2','fs2')


a = Dcel(address='file://fs.localhost', 
         service_class=BlackstrapFS)
b = Dcel(address='file://fs2.localhost', 
         service_class=BlackstrapFS)
c = Dcel(address=[a,b], 
         service_class=MulticelSeqFS)

In [9]:
for ea in c.listdir('/'):
    print(type(c[ea]))

print([c[ea] for ea in c.listdir('/')])

<class 'Dcel.Dcel'>
<class 'Dcel.Dcel'>
<class 'Dcel.Dcel'>
<class 'Dcel.Dcel'>
<class 'Dcel.Dcel'>
<class 'Dcel.Dcel'>
[<Dcel.Dcel object at 0x118d3ed60>, <Dcel.Dcel object at 0x118d3e280>, <Dcel.Dcel object at 0x118d3e070>, <Dcel.Dcel object at 0x118d3ed60>, <Dcel.Dcel object at 0x118d3e280>, <Dcel.Dcel object at 0x118d4a3a0>]


In [35]:
# glob proof of concept
# using MulticelSeqFS

def globall(b):
    res_list = [b[ea]
                for ea in b.listdir('/')]
    c = Dcel(res_list,
             service_class=MulticelSeqFS
            )
    return c

c = globall(a)
for ea in c.listdir('/'):
    print(c[ea].getinfo('.'))

<file 'apples.txt'>
<file 'cherries.html.txt'>
<file 'junk.txt'>
<file 'catamaran.txt'>
<file 'skiff.txt'>
<dir 'tools'>
<dir 'types'>
<dir 'etc'>
<dir 'var'>
<dir 'hosts'>


In [13]:
i = c.getinfo('boats', 
          namespaces=['dcel'])
print(i.raw)
b.service.url.netlock

{'basic': {'name': 'boats', 'is_dir': True}, 'hosts': ['fs.localhost', 'fs2.localhost']}


'fs2.localhost'